In [1]:
%pip install -qU langchain-pinecone langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 2.6 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
from pinecone import Pinecone, ServerlessSpec

pinecone_api_key = userdata.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

In [3]:
import time

index_name = "online-rag-project3"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

#
#
#index_name = "online-rag-project3"

#pc.create_index(
    #name=index_name,
    #dimension=768, # Replace with your model dimensions
    #metric="cosine", # Replace with your model metric
    #spec=ServerlessSpec(
        #cloud="aws",
        #region="us-east-1"
    #)
#)

#index = pc.Index(index_name)

In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [5]:
vector = embeddings.embed_query("I am building a RAG project")

In [6]:
vector[:3]

[0.008258451707661152, -0.05451526492834091, -0.04841499403119087]

In [7]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [9]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="As an open-source framework, LangChain benefits from community contributions, ensuring continuous improvement and adaptability to emerging AI advancements.",
    metadata={"source": "jonascleveland"},
)

document_10 = Document(
    page_content="LangChain offers a wide array of integrations with various systems and platforms, facilitating seamless incorporation of large language models into diverse applications.",
    metadata={"source": "encyclopedia"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['fc5d1e13-7283-4739-b6ce-61c57157b1aa',
 '64063255-818e-43b2-9220-f933e6ec93a7',
 '7e2b241d-e46f-40b8-9fa8-0877a7bc7bf3',
 '9b3dbc92-7ff3-42aa-a9e7-cf2ccc071d57',
 'cc243d49-aa83-4edf-a206-a214b2913db7',
 '3081233a-ad1b-4dfe-8810-6601219833f1',
 '212a6978-c429-446e-9b2d-5814456c446c',
 'f1663b33-223a-41f3-a303-e7bf93930612',
 '1d7407e4-389f-43fa-b3ce-26aeb3b272a4',
 '8175ce48-35ff-40e6-bdc7-656cd48f2f84']

In [12]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "encyclopedia"}
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* LangChain offers a wide array of integrations with various systems and platforms, facilitating seamless incorporation of large language models into diverse applications. [{'source': 'encyclopedia'}]


In [13]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=1, filter={"source": "jonascleveland"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.414939] As an open-source framework, LangChain benefits from community contributions, ensuring continuous improvement and adaptability to emerging AI advancements. [{'source': 'jonascleveland'}]


In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [15]:
def answer_to_user(query):
  #Vector Search
  vector_results = vector_store.similarity_search(query, k=3)

  #Pass Model with Vector results + User Query
  final_answer = llm.invoke(f"This is the user Query:{query}. Here are some references:{vector_results}")

  return final_answer.content

In [17]:
print(answer_to_user("what are the benefits of langchain that other competitors doesn't have? You can ignore the reference if the reference is not relevant."))

LangChain's key advantages over competitors, based on the provided information, are:

* **Open-source nature with community contributions:** This fosters continuous improvement, adaptability to new AI advancements, and potentially faster issue resolution compared to closed-source alternatives.
* **Wide array of integrations:**  LangChain connects with various systems and platforms, making it easier to incorporate large language models into diverse applications.  This suggests greater flexibility and potential for more complex workflows than competitors with fewer integrations.
